In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName('Spark Core').setMaster('local[*]')
sc = SparkContext(conf=conf)

In [4]:
rdd1 = sc.parallelize(range(5))
rdd1.flatMap((lambda e: range(e))).collect()

[0, 0, 1, 0, 1, 2, 0, 1, 2, 3]

In [ ]:
import json

sc.textFile("../user.json")\
    .map(lambda x: json.loads(x))\
    .collect()

In [11]:
sc.textFile("../user.json")\
    .map(lambda x: json.loads(x))\
    .map(lambda x:(x['name'], x['age']))\
    .collect()

[('张三1', 19),
 ('张三2', 18),
 ('张三3', 17),
 ('张三4', 19),
 ('张三5', 17),
 ('张三6', 18),
 ('张三7', 19)]

In [6]:
rdd2 = sc.parallelize([(1,1001),(2,1002),(3,1003),(4,1004),(5,1005)])
def pf(ins):
    arr = []
    for i in ins:
        arr.append(str(i[0]) + '-' + str(i[1]))
    return arr
rdd2.mapPartitions(pf).collect()

['1-1001', '2-1002', '3-1003', '4-1004', '5-1005']

In [7]:
rdd3 = sc.parallelize([(1,1001),(2,1002),(3,1003),(4,1004),(5,1005)])
def pf3(index,ins): #参数变化了，两个参数
  arr3 = []
  for i in ins:
    arr3.append(str(index)+":"+str(i[0]) + "-"+str(i[1]))
  return arr3
rdd3.mapPartitionsWithIndex(pf3).collect()


['1:1-1001', '3:2-1002', '4:3-1003', '6:4-1004', '7:5-1005']

In [22]:
rdd4 = sc.parallelize(range(5), 1)
# 期望次数为5
rdd4.sample(True, 5).collect()

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4]

In [16]:
rdd4.sample(False, 1).collect()


[0, 1, 2, 3, 4]

In [19]:
# 抽到每个元素的概率，可能为空，或者全部抽中
rdd4.sample(False, 0.5).collect()

[0, 2, 3, 4]

In [23]:
# 有放回
rdd4.takeSample(True, 3)

[3, 2, 0]

In [25]:
# 无放回
rdd4.takeSample(False, 5)

[1, 4, 3, 2, 0]

In [26]:
sc.range(1, 10).collect()


[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [27]:
# 合并元素，保留重复项
sc.range(1, 10).union(sc.range(5, 15)).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [28]:
# 求交集
sc.range(1, 10).intersection(sc.range(5, 10)).collect()

[5, 6, 7, 8, 9]

In [30]:
rdd5 = sc.parallelize([1,2,3,4,5,6,7,1,2,3,5,9,10])
rdd5.distinct().collect()

[1, 9, 2, 10, 3, 4, 5, 6, 7]

In [31]:
# 并行度为4，表示分区为4
rdd7 = sc.parallelize([(1,'a'),(2,'b'),(3,'c'),(4,'d'),(5,'e'),(6,'f'),(7,'g')],4)
# glom可将RDD中的元素按分区收集起来
rdd7.glom().collect()

[[(1, 'a')], [(2, 'b'), (3, 'c')], [(4, 'd'), (5, 'e')], [(6, 'f'), (7, 'g')]]

In [40]:
# 自定义分区，返回结果为整数，代表分区数
def df7(e):
    return e%3

rdd7.partitionBy(3, df7).glom().collect()

[[(3, 'c'), (4, 'd'), (7, 'g')], [(1, 'a'), (5, 'e')], [(2, 'b'), (6, 'f')]]

In [41]:
# 超过分区编号，多余的分区里无内容
rdd7.partitionBy(4, df7).glom().collect()

[[(4, 'd')], [(1, 'a'), (5, 'e')], [(2, 'b'), (6, 'f')], [(3, 'c'), (7, 'g')]]

In [42]:
# 会合并两个分区（猜测：两个内容少的分区）里的内容
rdd7.partitionBy(2, df7).glom().collect()

[[(2, 'b'), (4, 'd'), (6, 'f')], [(1, 'a'), (3, 'c'), (5, 'e'), (7, 'g')]]

In [43]:
rdd8 = sc.parallelize('ASDSFSDFSDASDFSDA')
rdd8.map(lambda x: (x, 1))\
    .reduceByKey(lambda v1, v2: v1+v2).collect()

[('A', 3), ('F', 3), ('S', 6), ('D', 5)]

In [44]:
rdd8.map(lambda x: (x, 1)).groupByKey().collect()

[('A', <pyspark.resultiterable.ResultIterable at 0x1ec81eab048>),
 ('F', <pyspark.resultiterable.ResultIterable at 0x1ec81eab2c8>),
 ('S', <pyspark.resultiterable.ResultIterable at 0x1ec82294c08>),
 ('D', <pyspark.resultiterable.ResultIterable at 0x1ec821da588>)]

In [73]:
rdd9 = sc.parallelize([("a",90),("a",60),("a",70),("b",96),("b",80),("b",70),("c",60),("c",80),("c",80)])
# 注意 （k,v）对中v的形式相同，如本例子中的new_v
rdd9.combineByKey(lambda new_v:(new_v, 1),
                  lambda v, new_v: (v[0] +new_v, v[1]+ 1),
                  lambda u1, u2: (u1[0]+ u2[0], u1[1] + u2[1])).collect()

[('a', (220, 3)), ('b', (246, 3)), ('c', (220, 3))]

In [53]:
rdd9.aggregateByKey((0,0),
                    lambda v, new_v: (v[0] +new_v, v[1]+ 1),
                    lambda u1, u2: (u1[0]+ u2[0], u1[1] + u2[1])).collect()


[('a', (220, 3)), ('b', (246, 3)), ('c', (220, 3))]

In [61]:
# 不同分区上相同的k进行操作
rdd9.foldByKey(0,
               lambda u1, u2: (u1+ u2)).collect()

[('a', 220), ('b', 246), ('c', 220)]

In [66]:
# numSlices 表示分区数
rdd10 = sc.range(1, 10, numSlices=3)
rdd10.sortBy(lambda e:e, ascending=False).collect()

[9, 8, 7, 6, 5, 4, 3, 2, 1]

In [67]:
rdd11 = sc.parallelize([(1,'a'),(3,'b'),(4,'d'),(2,'e'),(5,'c')])
rdd11.sortBy(lambda e:e[0],ascending=True).collect()


[(1, 'a'), (2, 'e'), (3, 'b'), (4, 'd'), (5, 'c')]

In [68]:
rdd11.sortBy(lambda e:e[1],ascending=True).collect()


[(1, 'a'), (3, 'b'), (5, 'c'), (4, 'd'), (2, 'e')]

In [74]:
rdd12_1 = sc.parallelize([(1,'aa'),(2,'bb'),(3,'cc')])
rdd12_2 = sc.parallelize([(1,'AA'),(2,'BB'),(3,'CC')])

# 根据key，将value组合一起
rdd12_1.join(rdd12_2).collect()

[(1, ('aa', 'AA')), (2, ('bb', 'BB')), (3, ('cc', 'CC'))]

In [75]:
sc.range(1, 10).subtract(sc.range(5, 15)).collect()

[1, 2, 3, 4]

In [76]:
rdd13 = sc.parallelize([(1,10001),(2,10002),(1,20001),(3,10003),(2,20002),(4,100004),(5,10005),(3,20003)])
rdd13.mapValues(lambda v: v+1).collect()

[(1, 10002),
 (2, 10003),
 (1, 20002),
 (3, 10004),
 (2, 20003),
 (4, 100005),
 (5, 10006),
 (3, 20004)]

In [77]:
# func函数聚集RDD中的所有元素，这个功能必须是可交换且可并联的.
sc.range(1,10).reduce(lambda t1,t2:t1+t2)

45

In [78]:
#显示在终端
sc.range(1,5).foreach(lambda e:print(e))

In [84]:
# 累加器
rdd1 = sc.parallelize("fsdasdaSfd")
sum = 0
def m(data):
    global sum
    if data == 'a':
        sum += 1
    print(sum)
rdd1.foreach(m)
print(sum)

0


In [81]:
rdd1 = sc.parallelize("fsdasdaSfd")
acc = sc.accumulator(0)
def m(data):
    if data == 'a':
        acc.add(1)
rdd1.foreach(m)
print(acc.value)


2


In [91]:
# 广播变量
common = [(1,3),(2,4),(3,6),(4,6),(5,6),(6,6)] #【这个变量的问题】
rdd = sc.parallelize([(1,"a"),(2,"b"),(3,"c"),(4,"d"),(5,"e"),(6,"f")])
def mf(e):
    for i in common:
        if i[0] == e[0]:
            return i[0], i[1], e[1]

rdd.map(mf).collect()

[(1, 3, 'a'), (2, 4, 'b'), (3, 6, 'c'), (4, 6, 'd'), (5, 6, 'e'), (6, 6, 'f')]

In [92]:
common1 = [(1,3),(2,4),(3,6),(4,6),(5,6),(6,6)] #【这个变量的问题】
broadcast = sc.broadcast(common1)
rdd1 = sc.parallelize([(1,"a"),(2,"b"),(3,"c"),(4,"d"),(5,"e"),(6,"f")])
def mf(e):
    for i in broadcast.value:
        if i[0] == e[0]:
            return i[0], i[1], e[1]

rdd1.map(mf).collect()

[(1, 3, 'a'), (2, 4, 'b'), (3, 6, 'c'), (4, 6, 'd'), (5, 6, 'e'), (6, 6, 'f')]

In [106]:
bigArr = [i for i in range(1, 1000)]
bd = sc.broadcast(bigArr)
rdd1 = sc.parallelize([30, 50000000, 70, 600000, 10, 20], 4)
rdd2 = rdd1.filter(lambda x: x in bd.value)
rdd2.collect()

[30, 70, 10, 20]

In [107]:
bigArr = [i for i in range(1, 1000)]
rdd1 = sc.parallelize([30, 50000000, 70, 600000, 10, 20], 4)
rdd2 = rdd1.filter(lambda x: x in bigArr)
rdd2.collect()

[30, 70, 10, 20]

In [105]:
print(bd.value.__contains__(30))



True
